In [ ]:
key = '################################################################'

In [ ]:
import requests
import json, os
import urllib.request
from shapely.geometry import Point, Polygon
from matplotlib import pyplot as plt
import shapely
import pickle
import random
import numpy as np
import gmaps
# make sure this points to the file the data is to be saved in
data_dir = "./data"

In [ ]:
def plot_map(poly_grid,Loc=[],restrict=[]):
    g_poly = []
    s_loc = []
    s_htext = []
    if len(restrict)==0 :
        restrict = list(poly_grid.keys())
    
    for k,v in poly_grid.items():
        if k in restrict :
            g_poly.append(gmaps.Polygon(
                                        list(v),
                                        stroke_color='blue',
                                        fill_color='red'
                                        ))
            s_loc.append((v[0][0],v[0][1]))
            s_htext.append(str(k))
    if len(Loc)!=0 :
        s_loc = []
        s_htext = []
        for k,v in Loc.items() :
            if k in restrict :
                s_loc += v
                s_htext += [str(k)+str(i) for i in v]
    fig = gmaps.figure(center=(20.5937,78.9629), zoom_level=4, map_type='TERRAIN')
    fig.add_layer(gmaps.drawing_layer(features=g_poly))
    fig.add_layer(gmaps.symbol_layer(s_loc,info_box_content=s_htext,scale=1))
    return fig

In [ ]:
indPolyGrid = pickle.load(open("indPolyGrid.pkl","rb"))
for i in indPolyGrid.values():
    plt.plot(i[:,1],i[:,0])
print(len(indPolyGrid))

In [ ]:
plot_map(indPolyGrid, restrict=[0])

In [ ]:
#search_grids = indPolyGrid.keys()
search_grids = list(range(50,60))
print("Searching grids : {}".format("All" if search_grids==indPolyGrid.keys() else search_grids))

In [ ]:
base = 'https://maps.googleapis.com/maps/api/streetview'
ext = '?size=600x300&location={}&fov=100&heading={}&radius={}&pitch=10&key={}'
print("Seacrching Grids: {}".format("All" if search_grids==indPolyGrid.keys() else search_grids))
for grid,coor in indPolyGrid.items():
    poly = Polygon(np.flip(coor))
    minx,miny,maxx,maxy = poly.bounds
    count = 0
    trials = 0
    loc_list = []
    if grid in search_grids :
        save_folder = data_dir+'/{}'.format(grid)
        if os.path.exists(save_folder)==False :
            os.mkdir(save_folder)
        loc_list = os.listdir(save_folder)
        print("################## Searching grid {} ###################".format(grid))
        print("Save folder {}".format(save_folder))
        print("Existing locations {}".format(loc_list))
        while count<20 and trials<10:
            point = Point(random.uniform(minx,maxx),random.uniform(miny,maxy))
            location = str(point.y)+','+str(point.x)
            if (poly.contains(point)) and (location not in loc_list):
                meta_url = base + '/metadata' + ext.format(location,0,1000,key)
                r = requests.get(meta_url).json()
                trials += 1
                print("Trial : {}, count : {}".format(trials,count))
                if r['status']=='OK' and poly.contains(Point(r['location']['lng'],r['location']['lat'])):
                    location = str(r['location']['lng'])+ ',' +str(r['location']['lat'])
                    if location not in loc_list :
                        print("Valid location found: {}".format(location))
                        loc_list.append(location)
                        save_file = save_folder + "/{}".format(location)
                        if os.path.exists(save_file)==False:
                            os.mkdir(save_file)
                        for heading in [0,90,180]:
                            img_url = base + ext.format(location,heading,1000,key)
                            urllib.request.urlretrieve(img_url,save_file+'/{}_{}.jpg'.format(heading,r['date']))
                            count += 1
                            trials = 0
                    else :
                        print("Failed trial {} location exist in folder".format(trials))
                        print("Location : {}".format(location))
        print(loc_list)
        print("No duplicates : {}".format(len(loc_list)==len(set(loc_list))))
        print("########## Final locations for grid {}: {} ############".format(grid, len(loc_list)))

In [ ]:
gds_loc = {}
for g in os.listdir(data_dir):
    gds_loc[int(g)] = []
    for loc in os.listdir(data_dir + "/{}".format(g)):
        loc_list = list(map(lambda x:float(x),loc.split(',')))
        gds_loc[int(g)].append(loc_list)


In [ ]:
grid = 51
print("Grid {} has {} locations".format(grid,len(gds_loc[grid])))
for lc in gds_loc[grid]:
    loc = ','.join([str(i) for i in lc])
    print(loc)
    for img in os.listdir(data_dir + "/{}/{}".format(grid,loc)):
        image = plt.imread(data_dir + "/{}/{}/{}".format(grid,loc,img))
        plt.imshow(image)
        plt.show()

In [ ]:
plot_map(indPolyGrid, Loc=gds_loc, restrict=[230])